In [2]:
!pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [128]:
import torch
import torch.nn as nn
import torch.optim as optim
import librosa
import os
from torch.utils.data import DataLoader
from pytorch_lightning import Trainer
from pytorch_lightning.cli import LightningCLI

In [97]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [98]:
model_path = '/content/drive/MyDrive/빅프/model/BEATs_iter3_plus_AS2M.pt'
checkpoint_path = '/content/drive/MyDrive/빅프/코드/lightning_logs/version_4/checkpoints/epoch=0-step=5072.ckpt'

In [138]:
import sys
sys.path.append('/content/drive/MyDrive/빅프/코드')
import BEATs

In [139]:
data_path = '/content/drive/MyDrive/빅프/test/'

In [162]:
from pytorch_lightning.trainer.trainer import Trainer
cp = torch.load(model_path)
cfg = BEATs.BEATsConfig(cp['cfg'])
model = BEATs.BEATs(cfg)
optimizer = optim.AdamW(model.parameters())

checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
for key in list(checkpoint['state_dict'].keys()):
    name = key[6:]
    checkpoint['state_dict'][name] = checkpoint['state_dict'].pop(key)
model.load_state_dict(checkpoint['state_dict'], strict=False)

model.eval()
with torch.no_grad():
  for data in os.listdir(data_path):
    audio_path = os.path.join(data_path, data)
    print(audio_path)
    sig, sr = librosa.load(audio_path, sr= 44100, mono = True)
    sig = librosa.util.fix_length(sig, size=5*sr)
    sig_t = torch.tensor(sig)
    sig_t = sig_t.reshape(-1,5*sr)
    fc = nn.Linear(model.cfg.encoder_embed_dim, model.cfg.predictor_class)
    output, _ = model.extract_features(sig_t)
    output= fc(output)
    output = output.mean(dim=1)
    print(output.argmax(1))

/content/drive/MyDrive/빅프/test/5.화재_6643_label.wav
tensor([476])
/content/drive/MyDrive/빅프/test/15.실내_532510_label.wav
tensor([444])
/content/drive/MyDrive/빅프/test/7.응급의료_344155_label.wav
tensor([285])
/content/drive/MyDrive/빅프/test/1.강제추행(성범죄)_129312_label.wav
tensor([255])
